# Back Propagation 

## Regression

In [15]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [16]:
df= pd.DataFrame([[8,8,4],[7,9,5],[6,10,6],[5,12,7]],columns=['cgpa','score','lpa'])
df

,cgpa,score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,7


### Own Custom Code

In [17]:
def initialize_parameters(layer_dims):
  
  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)         

  for l in range(1, L):

    parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l]))*0.1
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
      

  return parameters

In [18]:
def linear_forward(A_prev, W, b):
  
  Z = np.dot(W.T, A_prev) + b
  
  return Z

In [19]:
# Forward Prop
def L_layer_forward(X, parameters):

  A = X
  L = len(parameters) // 2                  # number of layers in the neural network
  
  for l in range(1, L+1):
    A_prev = A 
    Wl = parameters['W' + str(l)]
    bl = parameters['b' + str(l)]
    #print("A"+str(l-1)+": ", A_prev)
    #print("W"+str(l)+": ", Wl)
    #print("b"+str(l)+": ", bl)
    #print("--"*20)

    A = linear_forward(A_prev, Wl, bl)
    #print("A"+str(l)+": ", A)
    #print("**"*20)
          
  return A,A_prev

In [20]:
X = df[['cgpa', 'lpa']].values[0].reshape(2,1) # Shape(no of features, no. of training example)
y = df[['lpa']].values[0][0]

# Parameter initialization
parameters = initialize_parameters([2,2,1])

y_hat,A1 = L_layer_forward(X, parameters)

In [21]:
def update_parameters(parameters,y,y_hat,A1,X):
  parameters['W2'][0][0] = parameters['W2'][0][0] + (0.001 * 2 * (y - y_hat)*A1[0][0])
  parameters['W2'][1][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat)*A1[1][0])
  parameters['b2'][0][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat))

  parameters['W1'][0][0] = parameters['W1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[0][0])
  parameters['W1'][0][1] = parameters['W1'][0][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[1][0])
  parameters['b1'][0][0] = parameters['b1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0])

  parameters['W1'][1][0] = parameters['W1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[0][0])
  parameters['W1'][1][1] = parameters['W1'][1][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[1][0])
  parameters['b1'][1][0] = parameters['b1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0])

In [22]:
# epochs implementation

parameters = initialize_parameters([2,2,1])
epochs = 5

for i in range(epochs):

  Loss = []

  for j in range(df.shape[0]):

    X = df[['cgpa', 'lpa']].values[j].reshape(2,1) # Shape(no of features, no. of training example)
    y = df[['lpa']].values[j][0]

    # Parameter initialization


    y_hat,A1 = L_layer_forward(X,parameters)
    y_hat = y_hat[0][0]

    update_parameters(parameters,y,y_hat,A1,X)

    Loss.append((y-y_hat)**2)

  print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean())

parameters

Epoch -  1 Loss -  26.971452129354752
Epoch -  2 Loss -  23.1318366991317
Epoch -  3 Loss -  17.803992316459002
Epoch -  4 Loss -  11.467371382890294
Epoch -  5 Loss -  6.005790622768655


{'W1': array([[0.33731546, 0.32877191],
        [0.33135044, 0.32279316]]),
 'b1': array([[0.03884061],
        [0.0378453 ]]),
 'W2': array([[0.4513718 ],
        [0.43523592]]),
 'b2': array([[0.44210583]])}

### Using Tensorflow

In [23]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [24]:
model=Sequential()

model.add(Dense(2,activation='linear',input_dim=2))
model.add(Dense(1,activation='linear'))


In [25]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.get_weights()

[array([[-0.04327202,  0.9717976 ],
        [ 1.1353036 ,  0.31062937]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[1.0691024],
        [0.8199066]], dtype=float32),
 array([0.], dtype=float32)]

In [27]:
new_weights = [
    np.array([[0.1, 0.1], [0.1, 0.1]], dtype=np.float32),  
    np.array([0., 0.], dtype=np.float32),                  
    np.array([[0.1], [0.1]], dtype=np.float32),            
    np.array([0.], dtype=np.float32)                       
]
model.set_weights(new_weights)


In [28]:
model.get_weights()

[array([[0.1, 0.1],
        [0.1, 0.1]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[0.1],
        [0.1]], dtype=float32),
 array([0.], dtype=float32)]

In [29]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error',optimizer=optimizer)

In [30]:
x=df.drop(columns=['lpa'])
y=df.lpa
model.fit(x,y,epochs=75,verbose=1,batch_size=1)


Epoch 1/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 27.8792 
Epoch 2/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.5550 
Epoch 3/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.2559 
Epoch 4/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.8874 
Epoch 5/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 26.5666 
Epoch 6/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 26.1904 
Epoch 7/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.8089 
Epoch 8/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 25.4389 
Epoch 9/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.0542 
Epoch 10/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 24.6233 
Epoch 11/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 24.2110 
Epoch 12/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 23.7887 
Epoch 13/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 23.3874 
Epoch 14/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 22.9351 
Epoch 15/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 22.4542
Epoch 16/75
4/4 ━━━

In [31]:
model.get_weights()

[array([[0.37332368, 0.37332368],
        [0.36560133, 0.36560133]], dtype=float32),
 array([0.27217427, 0.27217427], dtype=float32),
 array([[0.37279445],
        [0.37279445]], dtype=float32),
 array([0.2048041], dtype=float32)]

## Classification

### Custom Code

In [32]:
df= pd.DataFrame([[8,8,1],[7,9,1],[6,10,0],[5,12,0]],columns=['cgpa','score','placed'])
df

,cgpa,score,placed
0,8,8,1
1,7,9,1
2,6,10,0
3,5,12,0


In [42]:
def initialize_parameters(layer_dims):
  
  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)         

  for l in range(1, L):

    parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l]))*0.1
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
      

  return parameters

In [43]:
# Utility Functions
def sigmoid(Z):
  
  A = 1/(1+np.exp(-Z))

  return A

In [44]:
def linear_forward(A_prev, W, b):
  
  Z = np.dot(W.T, A_prev) + b

  A = sigmoid(Z)
  
  return A

In [45]:
# L-layer feed forward

def L_layer_forward(X, parameters):

  A = X
  L = len(parameters) // 2                  # number of layers in the neural network
  
  for l in range(1, L+1):
    A_prev = A 
    Wl = parameters['W' + str(l)]
    bl = parameters['b' + str(l)]
    #print("A"+str(l-1)+": ", A_prev)
    #print("W"+str(l)+": ", Wl)
    #print("b"+str(l)+": ", bl)
    #print("--"*20)

    A = linear_forward(A_prev, Wl, bl)
    #print("A"+str(l)+": ", A)
    #print("**"*20)
          
  return A,A_prev


In [46]:
def update_parameters(parameters,y,y_hat,A1,X):
  parameters['W2'][0][0] = parameters['W2'][0][0] + (0.0001 * (y - y_hat)*A1[0][0])
  parameters['W2'][1][0] = parameters['W2'][1][0] + (0.0001 * (y - y_hat)*A1[1][0])
  parameters['b2'][0][0] = parameters['W2'][1][0] + (0.0001 * (y - y_hat))

  parameters['W1'][0][0] = parameters['W1'][0][0] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0])*X[0][0])
  parameters['W1'][0][1] = parameters['W1'][0][1] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0])*X[1][0])
  parameters['b1'][0][0] = parameters['b1'][0][0] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0]))

  parameters['W1'][1][0] = parameters['W1'][1][0] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0])*X[0][0])
  parameters['W1'][1][1] = parameters['W1'][1][1] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0])*X[1][0])
  parameters['b1'][1][0] = parameters['b1'][1][0] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0]))

In [48]:
# epochs implementation

parameters = initialize_parameters([2,2,1])
epochs = 50

for i in range(epochs):

  Loss = []

  for j in range(df.shape[0]):

    X = df[['cgpa', 'score']].values[j].reshape(2,1) # Shape(no of features, no. of training example)
    y = df[['placed']].values[j][0]

    # Parameter initialization


    y_hat,A1 = L_layer_forward(X,parameters)
    y_hat = y_hat[0][0]

    update_parameters(parameters,y,y_hat,A1,X)

    Loss.append(-y*np.log(y_hat) - (1-y)*np.log(1-y_hat))

  print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean())

parameters

Epoch -  1 Loss -  0.7135494021352312
Epoch -  2 Loss -  0.7023979867791279
Epoch -  3 Loss -  0.702393772800165
Epoch -  4 Loss -  0.7023895608076378
Epoch -  5 Loss -  0.7023853508005624
Epoch -  6 Loss -  0.7023811427779558
Epoch -  7 Loss -  0.7023769367388353
Epoch -  8 Loss -  0.7023727326822189
Epoch -  9 Loss -  0.7023685306071253
Epoch -  10 Loss -  0.7023643305125733
Epoch -  11 Loss -  0.7023601323975824
Epoch -  12 Loss -  0.7023559362611727
Epoch -  13 Loss -  0.7023517421023647
Epoch -  14 Loss -  0.7023475499201793
Epoch -  15 Loss -  0.7023433597136384
Epoch -  16 Loss -  0.7023391714817637
Epoch -  17 Loss -  0.702334985223578
Epoch -  18 Loss -  0.7023308009381042
Epoch -  19 Loss -  0.7023266186243662
Epoch -  20 Loss -  0.7023224382813881
Epoch -  21 Loss -  0.7023182599081943
Epoch -  22 Loss -  0.70231408350381
Epoch -  23 Loss -  0.7023099090672611
Epoch -  24 Loss -  0.7023057365975736
Epoch -  25 Loss -  0.7023015660937743
Epoch -  26 Loss -  0.7022973975548902

{'W1': array([[0.10003318, 0.09967885],
        [0.1000332 , 0.09967816]]),
 'b1': array([[-1.57001680e-05],
        [-1.57367755e-05]]),
 'W2': array([[0.098865  ],
        [0.09886551]]),
 'b2': array([[0.0988089]])}

### Tensorflow

In [33]:
model=Sequential()

model.add(Dense(2,activation='sigmoid',input_dim=2))
model.add(Dense(2,activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12 (48.00 B)

 Trainable params: 12 (48.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.get_weights()

[array([[-0.20757806, -0.62526464],
        [-0.8194369 , -0.2528258 ]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[ 0.27129495,  0.00948954],
        [-0.20333755, -1.1006511 ]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [38]:
# new_weights = [
#     np.array([[0.1, 0.1], [0.1, 0.1]], dtype=np.float32),  
#     np.array([0., 0.], dtype=np.float32),                 
#     np.array([[0.1], [0.1]], dtype=np.float32),            
#     np.array([0.], dtype=np.float32)                       
# ]
# model.set_weights(new_weights)

In [39]:
model.compile(loss='binary_crossentropy',optimizer=optimizer)

In [41]:
# x=df.drop(columns=['placed'])
# y=df.placed
# model.fit(x,y,epochs=50,verbose=1,batch_size=1)